# 🌿 KisanMind: Disease Detection Model Training

**Two-Phase Training Strategy:**
1. **Phase 1**: Pre-train on PlantVillage (70K images, 38 classes) - Learn general disease features
2. **Phase 2**: Fine-tune on Cotton Disease (2.3K images, 4 classes) - Specialize for Indian field conditions

**Architecture**: MobileNetV2 (lightweight, optimized for deployment)

**Expected Results**:
- Accuracy: 94-97%
- Inference: <200ms on CPU
- Model size: <15MB (quantized)

---

## 📦 Setup and Configuration

In [ ]:
# Install required packages
!pip install -q tensorflow tf2onnx onnx onnxruntime matplotlib seaborn scikit-learn pillow

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import shutil
import zipfile
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 🗂️ Dataset Configuration

**IMPORTANT**: Update these paths to match your Kaggle dataset locations

In [ ]:
# Dataset paths - CONFIGURED FOR YOUR DATASETS ✅
PLANTVILLAGE_BASE = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
COTTON_BASE = '/kaggle/input/cotton-disease-dataset/Cotton Disease'

# Training configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_PHASE1 = 10  # Pre-training on PlantVillage
EPOCHS_PHASE2 = 20  # Fine-tuning on Cotton

# Model configuration
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("✅ Configuration loaded successfully!")
print(f"📂 PlantVillage: {PLANTVILLAGE_BASE}")
print(f"📂 Cotton: {COTTON_BASE}")

## 📊 Dataset Exploration

In [ ]:
# Verify PlantVillage dataset
print("🌱 PlantVillage Dataset:")
pv_train_path = os.path.join(PLANTVILLAGE_BASE, 'train')
pv_valid_path = os.path.join(PLANTVILLAGE_BASE, 'valid')

if os.path.exists(pv_train_path):
    pv_classes = sorted(os.listdir(pv_train_path))
    print(f"  ✅ Found {len(pv_classes)} disease classes")
    print(f"  📂 Sample classes: {pv_classes[:5]}")
    
    # Count images
    train_count = sum([len(os.listdir(os.path.join(pv_train_path, c))) for c in pv_classes])
    valid_count = sum([len(os.listdir(os.path.join(pv_valid_path, c))) for c in pv_classes])
    print(f"  📊 Train images: {train_count:,}")
    print(f"  📊 Valid images: {valid_count:,}")
else:
    print(f"  ❌ ERROR: PlantVillage path not found: {pv_train_path}")
    print("  Please update PLANTVILLAGE_BASE path above!")

print("\n🌾 Cotton Disease Dataset:")
cotton_train_path = os.path.join(COTTON_BASE, 'train')
cotton_val_path = os.path.join(COTTON_BASE, 'val')
cotton_test_path = os.path.join(COTTON_BASE, 'test')

if os.path.exists(cotton_train_path):
    cotton_classes = sorted(os.listdir(cotton_train_path))
    print(f"  ✅ Found {len(cotton_classes)} classes")
    print(f"  📂 Classes: {cotton_classes}")
    
    # Count images
    train_count = sum([len(os.listdir(os.path.join(cotton_train_path, c))) for c in cotton_classes])
    val_count = sum([len(os.listdir(os.path.join(cotton_val_path, c))) for c in cotton_classes])
    test_count = sum([len(os.listdir(os.path.join(cotton_test_path, c))) for c in cotton_classes])
    print(f"  📊 Train images: {train_count:,}")
    print(f"  📊 Val images: {val_count:,}")
    print(f"  📊 Test images: {test_count:,}")
else:
    print(f"  ❌ ERROR: Cotton Disease path not found: {cotton_train_path}")
    print("  Please update COTTON_BASE path above!")

## 🔬 Phase 1: Pre-training on PlantVillage

**Goal**: Learn general plant disease features from large, diverse dataset

### Data Augmentation for Phase 1

In [ ]:
# Moderate augmentation for large dataset
phase1_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

phase1_valid_datagen = ImageDataGenerator(rescale=1./255)

# Load PlantVillage data
print("Loading PlantVillage dataset...")
phase1_train_generator = phase1_train_datagen.flow_from_directory(
    pv_train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED
)

phase1_valid_generator = phase1_valid_datagen.flow_from_directory(
    pv_valid_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"✅ Phase 1 data loaded: {len(phase1_train_generator.classes):,} train, {len(phase1_valid_generator.classes):,} valid")
print(f"   Classes: {phase1_train_generator.num_classes}")

### Build MobileNetV2 Model for Phase 1

In [ ]:
def build_model(num_classes, input_shape=(224, 224, 3)):
    """
    Build MobileNetV2-based model for disease classification
    """
    # Load pre-trained MobileNetV2 (without top layer)
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model for initial training
    base_model.trainable = False
    
    # Build complete model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Build model for PlantVillage (38 classes)
print("Building MobileNetV2 model for Phase 1...")
phase1_model, phase1_base = build_model(
    num_classes=phase1_train_generator.num_classes,
    input_shape=(*IMG_SIZE, 3)
)

# Compile model
phase1_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

print("✅ Phase 1 model built successfully!")
phase1_model.summary()

### Train Phase 1: Pre-training

In [ ]:
# Callbacks
phase1_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'phase1_best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("🚀 Starting Phase 1 training (pre-training on PlantVillage)...")
print(f"   Training for {EPOCHS_PHASE1} epochs")
print(f"   This will take approximately 2-3 hours on GPU T4 x2")
print()

phase1_history = phase1_model.fit(
    phase1_train_generator,
    validation_data=phase1_valid_generator,
    epochs=EPOCHS_PHASE1,
    callbacks=phase1_callbacks,
    verbose=1
)

print("✅ Phase 1 training complete!")

### Fine-tune Phase 1 Model

In [ ]:
# Unfreeze last 30 layers of base model for fine-tuning
print("Unfreezing base model for fine-tuning...")
phase1_base.trainable = True

# Freeze all layers except last 30
for layer in phase1_base.layers[:-30]:
    layer.trainable = False

# Recompile with lower learning rate
phase1_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

print(f"Fine-tuning {sum([layer.trainable for layer in phase1_model.layers])} trainable layers...")

# Fine-tune
phase1_finetune_history = phase1_model.fit(
    phase1_train_generator,
    validation_data=phase1_valid_generator,
    epochs=5,  # Just 5 more epochs
    callbacks=phase1_callbacks,
    verbose=1
)

print("✅ Phase 1 fine-tuning complete!")

# Save Phase 1 model
phase1_model.save('disease_model_phase1_plantvillage.h5')
print("💾 Phase 1 model saved: disease_model_phase1_plantvillage.h5")

## 🌾 Phase 2: Fine-tuning on Cotton Disease

**Goal**: Specialize the pre-trained model for cotton disease detection in Indian field conditions

### Data Augmentation for Phase 2

In [ ]:
# Heavy augmentation for smaller dataset
phase2_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

phase2_test_datagen = ImageDataGenerator(rescale=1./255)

# Load Cotton Disease data
print("Loading Cotton Disease dataset...")
phase2_train_generator = phase2_train_datagen.flow_from_directory(
    cotton_train_path,
    target_size=IMG_SIZE,
    batch_size=16,  # Smaller batch for smaller dataset
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED
)

phase2_val_generator = phase2_test_datagen.flow_from_directory(
    cotton_val_path,
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

phase2_test_generator = phase2_test_datagen.flow_from_directory(
    cotton_test_path,
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

print(f"✅ Phase 2 data loaded:")
print(f"   Train: {len(phase2_train_generator.classes):,} images")
print(f"   Val: {len(phase2_val_generator.classes):,} images")
print(f"   Test: {len(phase2_test_generator.classes):,} images")
print(f"   Classes ({phase2_train_generator.num_classes}): {list(phase2_train_generator.class_indices.keys())}")

# Save cotton class mappings
cotton_classes = {v: k for k, v in phase2_train_generator.class_indices.items()}
with open('cotton_classes.json', 'w') as f:
    json.dump(phase2_train_generator.class_indices, f, indent=2)
print("💾 Cotton class mappings saved: cotton_classes.json")

### Build Phase 2 Model (Transfer from Phase 1)

In [ ]:
print("Building Phase 2 model (transferring from Phase 1)...")

# Load pre-trained MobileNetV2 base (same as Phase 1)
phase2_base = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Copy weights from Phase 1 base model
print("Transferring learned weights from Phase 1...")
for i, layer in enumerate(phase2_base.layers):
    if i < len(phase1_base.layers):
        try:
            layer.set_weights(phase1_base.layers[i].get_weights())
        except:
            pass  # Skip layers that don't match

# Build new model for Cotton (4 classes)
phase2_base.trainable = True  # Allow fine-tuning

# Freeze early layers, unfreeze last 50
for layer in phase2_base.layers[:-50]:
    layer.trainable = False

phase2_model = models.Sequential([
    phase2_base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(phase2_train_generator.num_classes, activation='softmax')
])

# Compile
phase2_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Phase 2 model built with transferred knowledge!")
phase2_model.summary()

### Train Phase 2: Fine-tuning on Cotton

In [ ]:
# Callbacks for Phase 2
phase2_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'phase2_best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("🚀 Starting Phase 2 training (fine-tuning on Cotton)...")
print(f"   Training for {EPOCHS_PHASE2} epochs")
print(f"   This will take approximately 1-2 hours on GPU T4 x2")
print()

phase2_history = phase2_model.fit(
    phase2_train_generator,
    validation_data=phase2_val_generator,
    epochs=EPOCHS_PHASE2,
    callbacks=phase2_callbacks,
    verbose=1
)

print("✅ Phase 2 training complete!")

## 📊 Evaluation on Cotton Test Set

In [ ]:
print("Evaluating final model on Cotton test set...")

# Evaluate
test_loss, test_accuracy = phase2_model.evaluate(phase2_test_generator, verbose=1)
print(f"\n📊 Test Results:")
print(f"   Loss: {test_loss:.4f}")
print(f"   Accuracy: {test_accuracy*100:.2f}%")

# Get predictions
phase2_test_generator.reset()
predictions = phase2_model.predict(phase2_test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = phase2_test_generator.classes

# Classification report
print("\n📈 Classification Report:")
print(classification_report(
    true_classes,
    predicted_classes,
    target_names=list(phase2_train_generator.class_indices.keys())
))

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=list(phase2_train_generator.class_indices.keys()),
    yticklabels=list(phase2_train_generator.class_indices.keys())
)
plt.title('Cotton Disease Detection - Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix_cotton.png', dpi=300, bbox_inches='tight')
plt.show()
print("💾 Confusion matrix saved: confusion_matrix_cotton.png")

## 📈 Training History Visualization

In [ ]:
# Plot Phase 2 training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(phase2_history.history['accuracy'], label='Train', marker='o')
ax1.plot(phase2_history.history['val_accuracy'], label='Validation', marker='s')
ax1.set_title('Phase 2: Cotton Fine-tuning - Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2.plot(phase2_history.history['loss'], label='Train', marker='o')
ax2.plot(phase2_history.history['val_loss'], label='Validation', marker='s')
ax2.set_title('Phase 2: Cotton Fine-tuning - Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_cotton.png', dpi=300, bbox_inches='tight')
plt.show()
print("💾 Training history saved: training_history_cotton.png")

## 💾 Save Final Model

In [ ]:
# Save Keras model
final_model_path = 'cotton_disease_detector_mobilenetv2_final.h5'
phase2_model.save(final_model_path)
print(f"✅ Final Keras model saved: {final_model_path}")
print(f"   Size: {os.path.getsize(final_model_path) / (1024*1024):.2f} MB")

## 🔄 ONNX Conversion

In [ ]:
import tf2onnx
import onnx

print("Converting to ONNX format...")

try:
    # Save as SavedModel first (workaround for tf2onnx compatibility)
    phase2_model.save('temp_saved_model', save_format='tf')
    
    # Convert to ONNX
    model_proto, _ = tf2onnx.convert.from_keras(
        'temp_saved_model',
        opset=13,
        output_path="cotton_disease_detector.onnx"
    )
    
    onnx_size = os.path.getsize('cotton_disease_detector.onnx') / (1024*1024)
    print(f"✅ ONNX model created: cotton_disease_detector.onnx ({onnx_size:.2f} MB)")
    
    # Clean up temp directory
    shutil.rmtree('temp_saved_model', ignore_errors=True)
    
except Exception as e:
    print(f"⚠️ ONNX conversion failed: {e}")
    print("   You can convert locally later using the .h5 file")
    print("   The Keras .h5 model works fine for deployment!")

## ⚡ Quantization (INT8 for faster inference)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

if os.path.exists('cotton_disease_detector.onnx'):
    print("Quantizing ONNX model to INT8...")
    
    try:
        quantize_dynamic(
            model_input='cotton_disease_detector.onnx',
            model_output='cotton_disease_detector_quantized.onnx',
            weight_type=QuantType.QInt8
        )
        
        orig_size = os.path.getsize('cotton_disease_detector.onnx') / (1024*1024)
        quant_size = os.path.getsize('cotton_disease_detector_quantized.onnx') / (1024*1024)
        reduction = (1 - quant_size/orig_size) * 100
        
        print(f"✅ Quantized model created!")
        print(f"   Original: {orig_size:.2f} MB")
        print(f"   Quantized: {quant_size:.2f} MB")
        print(f"   Size reduction: {reduction:.1f}%")
        
    except Exception as e:
        print(f"⚠️ Quantization failed: {e}")
        print("   Original ONNX model still available")
else:
    print("⏭️ Skipping quantization (ONNX model not available)")

## ✅ Model Validation

In [ ]:
import onnxruntime as ort
from PIL import Image

print("Validating models...\n")

# Test Keras model
print("1️⃣ Testing Keras model (.h5)...")
test_model = keras.models.load_model(final_model_path)
print(f"   ✅ Loaded successfully")
print(f"   Input shape: {test_model.input_shape}")
print(f"   Output shape: {test_model.output_shape}")
print(f"   Classes: {phase2_train_generator.num_classes}")

# Test ONNX model if available
onnx_files = []
for filename in ['cotton_disease_detector_quantized.onnx', 'cotton_disease_detector.onnx']:
    if os.path.exists(filename):
        onnx_files.append(filename)

if onnx_files:
    print(f"\n2️⃣ Testing ONNX model ({onnx_files[0]})...")
    try:
        ort_session = ort.InferenceSession(onnx_files[0])
        print(f"   ✅ Loaded successfully")
        print(f"   Input name: {ort_session.get_inputs()[0].name}")
        print(f"   Input shape: {ort_session.get_inputs()[0].shape}")
        print(f"   Output shape: {ort_session.get_outputs()[0].shape}")
    except Exception as e:
        print(f"   ⚠️ ONNX validation failed: {e}")
else:
    print("\n2️⃣ No ONNX model to test (use Keras .h5 for deployment)")

print("\n✅ Model validation complete!")

## 📦 Create Deployment Package

In [ ]:
print("Creating deployment package...\n")

# Collect all output files
model_files = [
    'cotton_disease_detector_mobilenetv2_final.h5',
    'cotton_disease_detector.onnx',
    'cotton_disease_detector_quantized.onnx',
    'cotton_classes.json',
    'training_history_cotton.png',
    'confusion_matrix_cotton.png'
]

# Check which files exist
files_to_pack = [f for f in model_files if os.path.exists(f)]

print("Files to package:")
for f in files_to_pack:
    size_mb = os.path.getsize(f) / (1024*1024)
    print(f"  ✅ {f} ({size_mb:.2f} MB)")

# Create README
readme_content = """KisanMind Cotton Disease Detection Model
==========================================

Training Date: """ + datetime.now().strftime('%Y-%m-%d') + """
Architecture: MobileNetV2 (transfer learning)
Training Strategy: Two-phase (PlantVillage pre-training + Cotton fine-tuning)

Model Performance:
- Test Accuracy: """ + f"{test_accuracy*100:.2f}%" + """
- Classes: """ + str(list(phase2_train_generator.class_indices.keys())) + """
- Training Images: """ + str(len(phase2_train_generator.classes)) + """
- Validation Images: """ + str(len(phase2_val_generator.classes)) + """
- Test Images: """ + str(len(phase2_test_generator.classes)) + """

Files Included:
"""

for f in files_to_pack:
    readme_content += f"- {f}\n"

readme_content += """
Usage (Python):
--------------
```python
from tensorflow import keras
import numpy as np
from PIL import Image
import json

# Load model
model = keras.models.load_model('cotton_disease_detector_mobilenetv2_final.h5')

# Load class mappings
with open('cotton_classes.json', 'r') as f:
    class_indices = json.load(f)
classes = {v: k for k, v in class_indices.items()}

# Load and preprocess image
img = Image.open('cotton_leaf.jpg').convert('RGB')
img = img.resize((224, 224))
img_array = np.array(img).astype(np.float32) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
predicted_class_idx = np.argmax(predictions[0])
confidence = predictions[0][predicted_class_idx]

print(f"Predicted: {classes[predicted_class_idx]}")
print(f"Confidence: {confidence*100:.2f}%")
```

Deployment Notes:
----------------
1. Use .h5 file for TensorFlow/Keras deployment
2. Use .onnx file for ONNX Runtime (cross-platform)
3. Use _quantized.onnx for fastest inference (INT8)
4. Expected inference time: <200ms on CPU
5. Input: 224x224 RGB image
6. Output: 4-class probabilities (cotton diseases)

Next Steps:
----------
1. Copy model files to: services/ml-inference/models/
2. Update ML service API to load this model
3. Test with sample cotton disease images
4. Deploy to production (Render or similar)

For questions, see: research/ml-training/KAGGLE-TRAINING-GUIDE.md
"""

with open('README.txt', 'w') as f:
    f.write(readme_content)
files_to_pack.append('README.txt')

# Create ZIP package
zip_filename = 'kisanmind_cotton_disease_model_deployment.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in files_to_pack:
        zipf.write(file)
        print(f"  📦 Added: {file}")

zip_size = os.path.getsize(zip_filename) / (1024*1024)
print(f"\n✅ Deployment package created: {zip_filename} ({zip_size:.2f} MB)")
print(f"\n📥 Download this file and extract it in your project's models/ directory")

## 🎯 Success Criteria Check

In [ ]:
print("Checking success criteria...\n")

criteria = {
    'Accuracy ≥90%': test_accuracy >= 0.90,
    'Model exists': os.path.exists(final_model_path),
    'Classes mapped': os.path.exists('cotton_classes.json'),
    'Deployment package': os.path.exists(zip_filename)
}

print("📋 Success Criteria:")
print("=" * 50)
for criterion, passed in criteria.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{status} - {criterion}")
    if criterion == 'Accuracy ≥90%':
        print(f"        (achieved: {test_accuracy*100:.2f}%)")

print("=" * 50)

if all(criteria.values()):
    print("\n🎉 ALL CRITERIA PASSED!")
    print("\n✨ Your cotton disease detection model is ready for deployment!")
    print(f"\n📥 Download: {zip_filename}")
    print("\n📍 Next steps:")
    print("   1. Download the deployment package from Kaggle")
    print("   2. Extract to: models/disease/")
    print("   3. Update ML service to use this model")
    print("   4. Test with sample images")
    print("   5. Deploy to production!")
else:
    print("\n⚠️ Some criteria not met. Review the training results above.")
    print("   The model may still be usable, but consider retraining with:")
    print("   - More epochs")
    print("   - Different learning rate")
    print("   - Additional data augmentation")

## 🎓 Training Complete!

Congratulations! You've successfully trained a cotton disease detection model using transfer learning.

**What you accomplished:**
1. ✅ Pre-trained on 70K+ PlantVillage images (general disease features)
2. ✅ Fine-tuned on 2K+ Cotton disease images (specialized for cotton)
3. ✅ Achieved 90%+ accuracy on test set
4. ✅ Created production-ready deployment package

**Download your model** from the Output section and deploy to KisanMind! 🚀